In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
 
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["multidimensional"]

multi = mycol.find()
df = pd.DataFrame(list(multi))

dfmovimiento = pd.DataFrame(list(df.movimiento))
dffecha = pd.DataFrame(list(df.fecha))
dfdescuento = pd.DataFrame(list(df.descuento.dropna()))
dfdlc = pd.DataFrame(list(df.dlc.dropna())) #Probar con cambios
dfjuego = pd.DataFrame(list(df.juego.dropna())) #Probar con cambios
dfproveedor = pd.DataFrame(list(df.proveedor.dropna())) #Cambiar el drop
dfubicacion = pd.DataFrame(list(df.ubicacion.dropna())) #Cambiar el drop
dfusuario = pd.DataFrame(list(df.usuario.dropna())) #Cambiar el drop
dfdescuento


##Hacer dataframes con los siguientes caracteristicas
#Compras de juegos sin descuentos
#Compras de DLCs sin descuentos
#Compras de juegos con descuentos
#Compras de DLCs con descuentos
#Ventas de juegos sin descuentos
#Ventas de DLCs sin descuentos
#Ventas de juegos con descuentos
#Ventas de DLCs con descuentos

,fecha_fin,fecha_inicio,porcentaje,puntero_descuento
0,2019-10-11,2019-09-13,47,55
1,2019-06-14,2019-05-17,74,47
2,2018-04-04,2018-03-07,29,627
3,2018-04-04,2018-03-07,29,627
4,2018-04-04,2018-03-07,29,627
5,2018-04-04,2018-03-07,29,627
6,2018-04-04,2018-03-07,29,627
7,2019-08-10,2019-07-13,85,306
8,2018-04-04,2018-03-07,29,627
9,2018-04-04,2018-03-07,29,627
